Webscraping no Tesouro Direto
Ricardo B Garcia rbgarcia@gmail.com

Data de atualização: 06/2022

O problema

Extrair informações dos títulos públicos negociados no Tesouro Direto, tais como preço, rentabilidade anual (taxa de juros) e vencimento.
O Tesouro Direto tem uma API que fornece os dados em formato em JSON. O desafio é acessar a API e extrair esses dados
Os dados que queremos acessar estão em: https://www.tesourodireto.com.br/titulos/precos-e-taxas.htm
A API que fornece os dados que queremos acessar estão em: https://www.tesourodireto.com.br/json/br/com/b3/tesourodireto/service/api/treasurybondsinfo.json
Abra esse link e confira como esses dados estão em formato de difícil leitura, mas veremos como esses dados podem ser lidos usando Python.
Solução:

Aqui, utilizamos a biblioteca requests e seu método .get() para acessar a API e extrair os dados em JSON, e utilizamos a biblioteca Python de mesmo nome (json) para converter em dados do tipo dicionátio (dict) Python.
Mostramos as informações que são disponibilizadas nos diversos níveis de chaves.
Abaixo, temos o passo-a-passo da extração dos dados e no final criamos uma função que pode ser facilmente usada em códigos na área de finanças.
Bibliotecas utilizadas:

urllib3 (https://urllib3.readthedocs.io/en/stable/)
json (https://docs.python.org/3/library/json.html)
requests (https://requests.readthedocs.io/en/latest/)
pandas (https://pandas.pydata.org/)

Bibliotecas

In [1]:
# Desabilita o warning do request sem verificar certificado do servidor: 
# requests.get(url,verify=False) dispensa a verificaçao de certificado
import urllib3
urllib3.disable_warnings()

# Bibliotecas
import json # Utilizada para converter json em um dicionário em Python
import requests # Utilizada para fazer consultas em url
import pandas as pd # Utilizada para tabular dados

Carregando os dados do Tesouro Direto

In [2]:
# Acessando a API do Tesouro Direto
# Endereço
url = 'https://www.tesourodireto.com.br/json/br/com/b3/tesourodireto/service/api/treasurybondsinfo.json'

resp_requ = requests.get(url,verify=False) # Objeto contendo dados do request
resp_dict = resp_requ.json()               # Dados no formato de dicionário Python
print('Tipo resp_requ:', type(resp_requ))
print('Tipo resp_dict:', type(resp_dict))

Tipo resp_requ: <class 'requests.models.Response'>
Tipo resp_dict: <class 'dict'>


Obs.: Podemos criar diretamente o request em formato dicionário aplicando o método .json():

resp_dict = requests.get(url, verify=False).json()

Percorrendo as chaves do dicionário

In [3]:
print('Número de chaves em "resp_dict":', len(resp_dict))
for key in resp_dict:
    print('-' + key)

Número de chaves em "resp_dict": 4
-responseStatus
-responseStatusText
-statusInfo
-response


Quando o request foi bem sucedido, obtemos:

In [4]:

# responseStatus: 200
resp_dict['responseStatus']

# responseStatusText: 'success'
resp_dict['responseStatusText']

# statusInfo: 'OK'
resp_dict['statusInfo']


'OK'

Explorando a chave 'response'

In [5]:
print("Chaves dentro de resp_dict['response']:")
for key in resp_dict['response']: print('-' + key)

Chaves dentro de resp_dict['response']:
-BdTxTp
-TrsrBondMkt
-TrsrBdTradgList
-BizSts


Chave 'BdTxTp'

In [6]:
resp_dict['response']['BdTxTp']

{'cd': 0}

Chave TrsrBondMkt

In [7]:
print('Data e horário de abertura, fechamento e atualização; mercado Aberto ou Fechado:')
resp_dict['response']['TrsrBondMkt']

Data e horário de abertura, fechamento e atualização; mercado Aberto ou Fechado:


{'opngDtTm': '2025-05-22T09:25:00',
 'clsgDtTm': '2025-05-23T05:00:00',
 'qtnDtTm': '2025-05-22T09:21:19.22',
 'stsCd': 1,
 'sts': 'Aberto'}

In [8]:
# Para acessar o status do mercado, basta acrescentar a chave ['sts']
resp_dict['response']['TrsrBondMkt']['sts']

'Aberto'

Chave 'TrsrBdTradgList'

Como o nome diz, trata-se de uma lista contendo todos os títulos que podem ser comprados ou vendidos. Na seção seguinte iremos explorar essa lista.

In [9]:
print('Tamanho da lista "TrsrBdTradgList":', len(resp_dict['response']['TrsrBdTradgList']))

Tamanho da lista "TrsrBdTradgList": 58


Chave 'BizSts'

In [10]:
# Na chave 'BizSts' acessamos o horário de atualização dos preços
print('Data e horário da consulta:')
resp_dict['response']['BizSts']

Data e horário da consulta:


{'cd': None, 'dtTm': '2025-05-22 11:19:01.152'}

Explorando as informações dos títulos públicos no Tesouro Direto

- Na chave 'TrsrBdTradgList', acessamos a lista completa de títulos públicos do tesouro

- Listas em Python são acessadas por meio de índices: 0, 1, ...

- Somente parte dos títulos estão disponíveis para compra

- Outros títulos somente estão disponíveis para resgate

In [11]:
n_titulos = len(resp_dict['response']['TrsrBdTradgList'])
print('Número de títulos:', n_titulos)

Número de títulos: 58


Cada título é acessado por índice numérico seguido por 'TrsrBd'. Vejamos um exemplo:

In [12]:
resp_dict['response']['TrsrBdTradgList'][0]['TrsrBd']

{'cd': 170,
 'nm': 'Tesouro IPCA+ 2026',
 'featrs': 'Título pós-fixado, uma vez que parte do seu rendimento acompanha a variação da taxa de inflação (IPCA).\r\n',
 'mtrtyDt': '2026-08-15T00:00:00',
 'minInvstmtAmt': 0.0,
 'untrInvstmtVal': 0.0,
 'invstmtStbl': 'Aumenta o poder de compra do seu dinheiro, pois seu rendimento é composto por uma taxa de juros + a variação da inflação (IPCA). É mais interessante para quem pode deixar o dinheiro render até o vencimento do investimento, pois não paga juros semestrais. Em caso de resgate antecipado, o Tesouro Nacional garante sua recompra pelo seu valor de mercado.\r\n',
 'semiAnulIntrstInd': False,
 'rcvgIncm': 'Indicado para aqueles que querem realizar investimentos de longo prazo.\r\n',
 'anulInvstmtRate': 0.0,
 'anulRedRate': 9.34,
 'minRedQty': 0.01,
 'untrRedVal': 4038.35,
 'minRedVal': 40.38,
 'isinCd': 'BRSTNCNTB4W2',
 'FinIndxs': {'cd': 22, 'nm': 'IPCA'},
 'wdwlDt': None,
 'convDt': None,
 'BusSegmt': None,
 'amortQuotQty': 0}

Nota: Os títulos que saíram de negociação de compra apresentam a última chave 'wdwlDt', contendo a data em que o título foi removido (withdrawal) das negociações. Esses títulos são acessados através da aba "Resgatar" na página do Tesouro Direito: https://www.tesourodireto.com.br/titulos/precos-e-taxas.htm

Construindo uma tabela para listar as chaves e seu conteúdo

In [13]:
# Vamos listar cada chave contendo informações de um título público
itens = [] # Lista para armazenar as chaves
for item in resp_dict['response']['TrsrBdTradgList'][2]['TrsrBd']:
    itens.append(item) # Armazena a chave
    print(item)        # print da chave

cd
nm
featrs
mtrtyDt
minInvstmtAmt
untrInvstmtVal
invstmtStbl
semiAnulIntrstInd
rcvgIncm
anulInvstmtRate
anulRedRate
minRedQty
untrRedVal
minRedVal
isinCd
FinIndxs
wdwlDt
convDt
BusSegmt
amortQuotQty


In [14]:
# Listando o conteúdo de um dos títulos
conteudos = [] # Lista para armazenar o conteúdo de cada chave
for item in itens: # Para cada chave em itens, acesse seu conteúdo:
    conteudo = resp_dict['response']['TrsrBdTradgList'][17]['TrsrBd'][item]
    conteudos.append(conteudo) # armazena

In [15]:
# Criando uma legenda para o código das chaves
description = ['Code',
         'Name', 
         'Features', 
         'Maturity date',
         'Min. Inv. Amount',
         'Unitary Value',
         'Inv. Suitability',
         'Semi annual interest pay',
         'Recomendation',
         'Anual interest rate',
         'Anual interest rate (sell)',
         'Min. sell amount',
         'Unitary sell value (sell)',
         'Min. sell value',
         'ISIN code',
         'Indexation'        
]

In [16]:
# Criando uma descrição das chaves
descricao = ['Código',
         'Nome', 
         'Características', 
         'Vencimento',
         'Investimento mínimo',
         'Preço unitário',
         'Adequação',
         'Juros semestrais',
         'Recomendação',
         'Taxa de juros anual',
         'Taxa de juros (Resgate)',
         'Quantidade mínima (Resgate)',
         'Preço unitário (Resgate)',
         'Preço mínimo (Resgate)',
         'Código ISIN do título',
         'Indexação'        
]

A tabela abaixo sumariza as informações que podemos obter

In [17]:
# Tabelando as informações
min_len = min(len(itens), len(description), len(descricao), len(conteudos))
df_info_titulos = pd.DataFrame()
df_info_titulos['Chave'] = itens[:min_len]
df_info_titulos['Legenda'] = description[:min_len]
df_info_titulos['Descrição'] = descricao[:min_len]
df_info_titulos['Exemplo'] = conteudos[:min_len]
display(df_info_titulos)

,Chave,Legenda,Descrição,Exemplo
0,cd,Code,Código,93
1,nm,Name,Nome,Tesouro IPCA+ com Juros Semestrais 2045
2,featrs,Features,Características,"Título pós-fixado, uma vez que parte do seu re..."
3,mtrtyDt,Maturity date,Vencimento,2045-05-15T00:00:00
4,minInvstmtAmt,Min. Inv. Amount,Investimento mínimo,39.73
5,untrInvstmtVal,Unitary Value,Preço unitário,3973.75
6,invstmtStbl,Inv. Suitability,Adequação,"Aumenta o poder de compra do seu dinheiro, poi..."
7,semiAnulIntrstInd,Semi annual interest pay,Juros semestrais,True
8,rcvgIncm,Recomendation,Recomendação,None
9,anulInvstmtRate,Anual interest rate,Taxa de juros anual,7.21


Lista dos títulos disponíveis

In [18]:
print('\033[1m'+'Títulos do Tesouro:'+'\033[0m')
for i in range(n_titulos):
    nome = resp_dict['response']['TrsrBdTradgList'][i]['TrsrBd']['nm']
    print('-' + nome)

Títulos do Tesouro:
-Tesouro IPCA+ 2026
-Tesouro IPCA+ 2029
-Tesouro IPCA+ 2035
-Tesouro IPCA+ 2040
-Tesouro IPCA+ 2045
-Tesouro IPCA+ 2050
-Tesouro Prefixado com Juros Semestrais 2027
-Tesouro Prefixado com Juros Semestrais 2029
-Tesouro Prefixado com Juros Semestrais 2031
-Tesouro Prefixado com Juros Semestrais 2033
-Tesouro Prefixado com Juros Semestrais 2035
-Tesouro IGPM+ com Juros Semestrais 2031
-Tesouro IPCA+ com Juros Semestrais 2026
-Tesouro IPCA+ com Juros Semestrais 2030
-Tesouro IPCA+ com Juros Semestrais 2032
-Tesouro IPCA+ com Juros Semestrais 2035
-Tesouro IPCA+ com Juros Semestrais 2040
-Tesouro IPCA+ com Juros Semestrais 2045
-Tesouro IPCA+ com Juros Semestrais 2050
-Tesouro IPCA+ com Juros Semestrais 2055
-Tesouro IPCA+ com Juros Semestrais 2060
-Tesouro Prefixado 2026
-Tesouro Prefixado 2027
-Tesouro Prefixado 2028
-Tesouro Prefixado 2029
-Tesouro Prefixado 2031
-Tesouro Prefixado 2032
-Tesouro Selic 2026
-Tesouro Selic 2027
-Tesouro Selic 2028
-Tesouro Selic 2029
-

Nota: Os títulos em negociação possuem valor de compra 'untrInvstmtVal', a qual é zero para os títulos somente disponíveis para resgate. Podemos usar esse valor como critério para filtrar os títulos.

In [19]:
print('\033[1m'+'Títulos em negociação:'+'\033[0m')
for i in range(n_titulos):
    nome = resp_dict['response']['TrsrBdTradgList'][i]['TrsrBd']['nm']
    valor = resp_dict['response']['TrsrBdTradgList'][i]['TrsrBd']['untrInvstmtVal']
    if valor != 0:
        print('-' + nome)

Títulos em negociação:
-Tesouro IPCA+ 2029
-Tesouro IPCA+ 2040
-Tesouro IPCA+ 2050
-Tesouro Prefixado com Juros Semestrais 2035
-Tesouro IPCA+ com Juros Semestrais 2035
-Tesouro IPCA+ com Juros Semestrais 2045
-Tesouro IPCA+ com Juros Semestrais 2060
-Tesouro Prefixado 2028
-Tesouro Prefixado 2032
-Tesouro Selic 2028
-Tesouro Selic 2031
-Tesouro Renda+ Aposentadoria Extra 2030
-Tesouro Renda+ Aposentadoria Extra 2035
-Tesouro Renda+ Aposentadoria Extra 2040
-Tesouro Renda+ Aposentadoria Extra 2045
-Tesouro Renda+ Aposentadoria Extra 2050
-Tesouro Renda+ Aposentadoria Extra 2055
-Tesouro Renda+ Aposentadoria Extra 2060
-Tesouro Renda+ Aposentadoria Extra 2065
-Tesouro Educa+ 2026
-Tesouro Educa+ 2027
-Tesouro Educa+ 2028
-Tesouro Educa+ 2029
-Tesouro Educa+ 2030
-Tesouro Educa+ 2031
-Tesouro Educa+ 2032
-Tesouro Educa+ 2033
-Tesouro Educa+ 2034
-Tesouro Educa+ 2035
-Tesouro Educa+ 2036
-Tesouro Educa+ 2037
-Tesouro Educa+ 2038
-Tesouro Educa+ 2039
-Tesouro Educa+ 2040
-Tesouro Educa+ 20

In [20]:
print('\033[1m'+'Títulos somente disponíveis para resgate:'+'\033[0m')
for i in range(n_titulos):
    nome = resp_dict['response']['TrsrBdTradgList'][i]['TrsrBd']['nm']
    valor = resp_dict['response']['TrsrBdTradgList'][i]['TrsrBd']['untrInvstmtVal']
    if valor == 0:
        print('-' + nome)

Títulos somente disponíveis para resgate:
-Tesouro IPCA+ 2026
-Tesouro IPCA+ 2035
-Tesouro IPCA+ 2045
-Tesouro Prefixado com Juros Semestrais 2027
-Tesouro Prefixado com Juros Semestrais 2029
-Tesouro Prefixado com Juros Semestrais 2031
-Tesouro Prefixado com Juros Semestrais 2033
-Tesouro IGPM+ com Juros Semestrais 2031
-Tesouro IPCA+ com Juros Semestrais 2026
-Tesouro IPCA+ com Juros Semestrais 2030
-Tesouro IPCA+ com Juros Semestrais 2032
-Tesouro IPCA+ com Juros Semestrais 2040
-Tesouro IPCA+ com Juros Semestrais 2050
-Tesouro IPCA+ com Juros Semestrais 2055
-Tesouro Prefixado 2026
-Tesouro Prefixado 2027
-Tesouro Prefixado 2029
-Tesouro Prefixado 2031
-Tesouro Selic 2026
-Tesouro Selic 2027
-Tesouro Selic 2029


Obtendo tabela de preços

In [21]:
# Listas para armezar informações dos títulos
tipos = []
nomes = []
tx_compras = []
tx_vendas = []
p_compras = []
p_vendas = []
vencimentos = []


for i in range(n_titulos):
    tipos.append(resp_dict['response']['TrsrBdTradgList'][i]['TrsrBd']['FinIndxs']['nm'])      # nome do indexador 
    nomes.append(resp_dict['response']['TrsrBdTradgList'][i]['TrsrBd']['nm'])                  # nome do título
    vencimentos.append(resp_dict['response']['TrsrBdTradgList'][i]['TrsrBd']['mtrtyDt'])       # vencimento
    tx_compras.append(resp_dict['response']['TrsrBdTradgList'][i]['TrsrBd']['anulInvstmtRate'])# Taxa de juros (compra)
    p_compras.append(resp_dict['response']['TrsrBdTradgList'][i]['TrsrBd']['untrInvstmtVal'])  # Preço de compra
    tx_vendas.append(resp_dict['response']['TrsrBdTradgList'][i]['TrsrBd']['anulRedRate'])     # Taxa de juros (venda)
    p_vendas.append(resp_dict['response']['TrsrBdTradgList'][i]['TrsrBd']['untrRedVal'])       # Preço de venda

In [22]:
# Tabelando os dados em um dataframe
df = pd.DataFrame()
df['Tipo'] = tipos
df['Título'] = nomes
df['Vencimento'] = pd.to_datetime(vencimentos)
df['Rentabilidade (Compra)'] = tx_compras
df['Preço R$ (Compra)'] = p_compras    
df['Rentabilidade (Venda)'] = tx_vendas
df['Preço R$ (Venda)'] = p_vendas  
display(df)

,Tipo,Título,Vencimento,Rentabilidade (Compra),Preço R$ (Compra),Rentabilidade (Venda),Preço R$ (Venda)
0,IPCA,Tesouro IPCA+ 2026,2026-08-15,0.0000,0.00,9.3400,4038.35
1,IPCA,Tesouro IPCA+ 2029,2029-05-15,7.4100,3401.66,7.5300,3385.31
2,IPCA,Tesouro IPCA+ 2035,2035-05-15,0.0000,0.00,7.3600,2229.66
3,IPCA,Tesouro IPCA+ 2040,2040-08-15,7.0900,1597.89,7.2100,1570.38
4,IPCA,Tesouro IPCA+ 2045,2045-05-15,0.0000,0.00,7.2700,1118.82
5,IPCA,Tesouro IPCA+ 2050,2050-08-15,6.9900,828.62,7.1100,805.33
6,PREFIXADO,Tesouro Prefixado com Juros Semestrais 2027,2027-01-01,0.0000,0.00,14.1600,982.51
7,PREFIXADO,Tesouro Prefixado com Juros Semestrais 2029,2029-01-01,0.0000,0.00,13.9700,934.99
8,PREFIXADO,Tesouro Prefixado com Juros Semestrais 2031,2031-01-01,0.0000,0.00,14.2800,885.97
9,PREFIXADO,Tesouro Prefixado com Juros Semestrais 2033,2033-01-01,0.0000,0.00,14.2300,855.34


Obtendo tabelas menores

Podemos estar interessados somente em parte dos dados: compra ou resgate/venda, assim como em um tipo específico de título (selic, IPCA, prefixado)

In [23]:
# Operação pode ser de compra ou venda/resgate
# Se solicitamos compra, somente vemos os títulos disponíveis para compra
op = 'C' # compra 'C', venda 'V' ou '' sem restrições (exibe compra e venda)

# Tipo de título pode ser SELIC, PREFIXADO ou IPCA/IGPM
tp = 'S' # selic 'S', prefixado 'P', ipca 'I' ou '' exibe todos

In [24]:
# Ajuste dos parâmetros
op = '' # String vazia: sem filtro de compra ou venda
tp = '' # String vazia: sem filtro por tipo de título

# Abaixo, criamos um DF2 com informações filtradas
# Fazemos primeiro a seleção pelo tipo de título
if tp == 'S':
    df2 = df.iloc[df[df['Tipo']=='SELIC'].index] # Selecionamos pelos índices de interesse
elif tp == 'P':
    df2 = df.iloc[df[df['Tipo']=='PREFIXADO'].index] # Selecionamos pelos índices de interesse
elif tp == 'I':
    df2 = df.iloc[df[df['Tipo']=='IPCA'].index] # Selecionamos pelos índices de interesse
else:
    df2 = df.copy()
    
# Fazemos a seleção segundo compra ou venda
if op == 'C':
    df2 = df2[df2['Preço R$ (Compra)']!=0].drop(df2.columns[[5, 6]],axis=1) # exclusão colunas de venda
    display(df2)
elif op == 'V':
    df2 = df2.drop(df2.columns[[3, 4]],axis=1) # Exclusão colunas de compra
    display(df2)
else:
    display(df2)

,Tipo,Título,Vencimento,Rentabilidade (Compra),Preço R$ (Compra),Rentabilidade (Venda),Preço R$ (Venda)
0,IPCA,Tesouro IPCA+ 2026,2026-08-15,0.0000,0.00,9.3400,4038.35
1,IPCA,Tesouro IPCA+ 2029,2029-05-15,7.4100,3401.66,7.5300,3385.31
2,IPCA,Tesouro IPCA+ 2035,2035-05-15,0.0000,0.00,7.3600,2229.66
3,IPCA,Tesouro IPCA+ 2040,2040-08-15,7.0900,1597.89,7.2100,1570.38
4,IPCA,Tesouro IPCA+ 2045,2045-05-15,0.0000,0.00,7.2700,1118.82
5,IPCA,Tesouro IPCA+ 2050,2050-08-15,6.9900,828.62,7.1100,805.33
6,PREFIXADO,Tesouro Prefixado com Juros Semestrais 2027,2027-01-01,0.0000,0.00,14.1600,982.51
7,PREFIXADO,Tesouro Prefixado com Juros Semestrais 2029,2029-01-01,0.0000,0.00,13.9700,934.99
8,PREFIXADO,Tesouro Prefixado com Juros Semestrais 2031,2031-01-01,0.0000,0.00,14.2800,885.97
9,PREFIXADO,Tesouro Prefixado com Juros Semestrais 2033,2033-01-01,0.0000,0.00,14.2300,855.34


Exemplificando como funciona a filtragem por índice

In [25]:
# Filtra os dados pela coluna tipo e seleciona os índices
df[df['Tipo']=='SELIC'].index

Index([27, 28, 29, 30, 31], dtype='int64')

In [26]:
# Seleciona o df utilizando os índices filtrados
df.iloc[df[df['Tipo']=='SELIC'].index]

,Tipo,Título,Vencimento,Rentabilidade (Compra),Preço R$ (Compra),Rentabilidade (Venda),Preço R$ (Venda)
27,SELIC,Tesouro Selic 2026,2026-03-01,0.0000,0.00,0.0141,16596.74
28,SELIC,Tesouro Selic 2027,2027-03-01,0.0000,0.00,0.0475,16584.67
29,SELIC,Tesouro Selic 2028,2028-03-01,0.0602,16580.05,0.0702,16566.37
30,SELIC,Tesouro Selic 2029,2029-03-01,0.0000,0.00,0.1088,16531.01
31,SELIC,Tesouro Selic 2031,2031-03-01,0.1116,16501.74,0.1216,16483.21


In [27]:
df[df['Tipo']=='PREFIXADO'].index

Index([6, 7, 8, 9, 10, 21, 22, 23, 24, 25, 26], dtype='int64')

In [28]:
# Seleciona o df utilizando os índices filtrados
df.iloc[df[df['Tipo']=='PREFIXADO'].index]

,Tipo,Título,Vencimento,Rentabilidade (Compra),Preço R$ (Compra),Rentabilidade (Venda),Preço R$ (Venda)
6,PREFIXADO,Tesouro Prefixado com Juros Semestrais 2027,2027-01-01,0.00,0.00,14.16,982.51
7,PREFIXADO,Tesouro Prefixado com Juros Semestrais 2029,2029-01-01,0.00,0.00,13.97,934.99
8,PREFIXADO,Tesouro Prefixado com Juros Semestrais 2031,2031-01-01,0.00,0.00,14.28,885.97
9,PREFIXADO,Tesouro Prefixado com Juros Semestrais 2033,2033-01-01,0.00,0.00,14.23,855.34
10,PREFIXADO,Tesouro Prefixado com Juros Semestrais 2035,2035-01-01,14.25,831.02,14.37,825.61
21,PREFIXADO,Tesouro Prefixado 2026,2026-01-01,0.00,0.00,14.81,917.55
22,PREFIXADO,Tesouro Prefixado 2027,2027-01-01,0.00,0.00,14.16,807.86
23,PREFIXADO,Tesouro Prefixado 2028,2028-01-01,13.66,716.54,13.78,714.21
24,PREFIXADO,Tesouro Prefixado 2029,2029-01-01,0.00,0.00,13.93,626.03
25,PREFIXADO,Tesouro Prefixado 2031,2031-01-01,0.00,0.00,14.16,477.64


Exemplificando como funciona a filtragem por coluna

In [29]:
# Para mostrar somente os preços de compra, drop nas colunas de índice 5 e 6
df.drop(df.columns[[5, 6]], axis=1).head()

,Tipo,Título,Vencimento,Rentabilidade (Compra),Preço R$ (Compra)
0,IPCA,Tesouro IPCA+ 2026,2026-08-15,0.00,0.00
1,IPCA,Tesouro IPCA+ 2029,2029-05-15,7.41,3401.66
2,IPCA,Tesouro IPCA+ 2035,2035-05-15,0.00,0.00
3,IPCA,Tesouro IPCA+ 2040,2040-08-15,7.09,1597.89
4,IPCA,Tesouro IPCA+ 2045,2045-05-15,0.00,0.00


In [30]:
# Para mostrar somente os preços de venda, drop nas colunas de índice 3 e 4
df.drop(df.columns[[3, 4]], axis=1).head()

,Tipo,Título,Vencimento,Rentabilidade (Venda),Preço R$ (Venda)
0,IPCA,Tesouro IPCA+ 2026,2026-08-15,9.34,4038.35
1,IPCA,Tesouro IPCA+ 2029,2029-05-15,7.53,3385.31
2,IPCA,Tesouro IPCA+ 2035,2035-05-15,7.36,2229.66
3,IPCA,Tesouro IPCA+ 2040,2040-08-15,7.21,1570.38
4,IPCA,Tesouro IPCA+ 2045,2045-05-15,7.27,1118.82


Exemplificando como funciona a filtragem pelo preço de compra

In [31]:
# Somente os títulos em negociação possuem um preço de compra
df[df['Preço R$ (Compra)'] != 0]

,Tipo,Título,Vencimento,Rentabilidade (Compra),Preço R$ (Compra),Rentabilidade (Venda),Preço R$ (Venda)
1,IPCA,Tesouro IPCA+ 2029,2029-05-15,7.4100,3401.66,7.5300,3385.31
3,IPCA,Tesouro IPCA+ 2040,2040-08-15,7.0900,1597.89,7.2100,1570.38
5,IPCA,Tesouro IPCA+ 2050,2050-08-15,6.9900,828.62,7.1100,805.33
10,PREFIXADO,Tesouro Prefixado com Juros Semestrais 2035,2035-01-01,14.2500,831.02,14.3700,825.61
15,IPCA,Tesouro IPCA+ com Juros Semestrais 2035,2035-05-15,7.3100,4126.68,7.4300,4090.61
17,IPCA,Tesouro IPCA+ com Juros Semestrais 2045,2045-05-15,7.2100,3973.75,7.3300,3922.20
20,IPCA,Tesouro IPCA+ com Juros Semestrais 2060,2060-08-15,7.1400,3955.53,7.2600,3894.81
23,PREFIXADO,Tesouro Prefixado 2028,2028-01-01,13.6600,716.54,13.7800,714.21
26,PREFIXADO,Tesouro Prefixado 2032,2032-01-01,14.0900,420.31,14.2100,417.19
29,SELIC,Tesouro Selic 2028,2028-03-01,0.0602,16580.05,0.0702,16566.37


Considerações:

Utilizando 2 parâmetros (operação e tipo), podemos selecionar se desejamos consultar informações de compra e/ou resgate, assim como o tipo de título. Veja abaixo uma função que encapsula todos os procedimentos.

Função para consultar o Tesouro Direto

In [32]:
def consultaTD(op,tp):
    """
    op:'C','V','' para Compra, Venda ou ambos
    tp:'S','P','I', '' para Selic, Prefixado, IPCA ou todos
    """
    op = op
    tp = tp
    
    url = 'https://www.tesourodireto.com.br/json/br/com/b3/tesourodireto/service/api/treasurybondsinfo.json'
    resp_dict = requests.get(url,verify=False).json()
    
    n_titulos = len(resp_dict['response']['TrsrBdTradgList'])
    
    tipos = []
    nomes = []
    tx_compras = []
    tx_vendas = []
    p_compras = []
    p_vendas = []
    vencimentos = []
    
    
    for i in range(n_titulos):
        tipos.append(resp_dict['response']['TrsrBdTradgList'][i]['TrsrBd']['FinIndxs']['nm']) 
        nomes.append(resp_dict['response']['TrsrBdTradgList'][i]['TrsrBd']['nm'])
        vencimentos.append(resp_dict['response']['TrsrBdTradgList'][i]['TrsrBd']['mtrtyDt'])
        tx_compras.append(resp_dict['response']['TrsrBdTradgList'][i]['TrsrBd']['anulInvstmtRate'])
        p_compras.append(resp_dict['response']['TrsrBdTradgList'][i]['TrsrBd']['untrInvstmtVal'])
        tx_vendas.append(resp_dict['response']['TrsrBdTradgList'][i]['TrsrBd']['anulRedRate'])
        p_vendas.append(resp_dict['response']['TrsrBdTradgList'][i]['TrsrBd']['untrRedVal']) 
    
    
    df = pd.DataFrame()
    df['Tipo'] = tipos
    df['Título'] = nomes
    df['Vencimento'] = pd.to_datetime(vencimentos)
    df['Rentabilidade (Compra)'] = tx_compras
    df['Preço R$ (Compra)'] = p_compras    
    df['Rentabilidade (Venda)'] = tx_vendas
    df['Preço R$ (Venda)'] = p_vendas
    
    if tp == 'S':
        df = df.iloc[df[df['Tipo']=='SELIC'].index]
    elif tp == 'P':
        df = df.iloc[df[df['Tipo']=='PREFIXADO'].index]
    elif tp == 'I':
        df = df.iloc[df[df['Tipo']=='IPCA'].index]
    else:
        df = df
        
    if op == 'C':
        return df[df['Preço R$ (Compra)']!=0].drop(df.columns[[5, 6]],axis=1)
    elif op == 'V':
        return df.drop(df.columns[[3, 4]],axis=1)
    else:
        return df

Exemplo de consultas

In [33]:
consultaTD('C','')

,Tipo,Título,Vencimento,Rentabilidade (Compra),Preço R$ (Compra)
1,IPCA,Tesouro IPCA+ 2029,2029-05-15,7.4100,3401.66
3,IPCA,Tesouro IPCA+ 2040,2040-08-15,7.0900,1597.89
5,IPCA,Tesouro IPCA+ 2050,2050-08-15,6.9900,828.62
10,PREFIXADO,Tesouro Prefixado com Juros Semestrais 2035,2035-01-01,14.2500,831.02
15,IPCA,Tesouro IPCA+ com Juros Semestrais 2035,2035-05-15,7.3100,4126.68
17,IPCA,Tesouro IPCA+ com Juros Semestrais 2045,2045-05-15,7.2100,3973.75
20,IPCA,Tesouro IPCA+ com Juros Semestrais 2060,2060-08-15,7.1400,3955.53
23,PREFIXADO,Tesouro Prefixado 2028,2028-01-01,13.6600,716.54
26,PREFIXADO,Tesouro Prefixado 2032,2032-01-01,14.0900,420.31
29,SELIC,Tesouro Selic 2028,2028-03-01,0.0602,16580.05


In [34]:
consultaTD('C','I')

,Tipo,Título,Vencimento,Rentabilidade (Compra),Preço R$ (Compra)
1,IPCA,Tesouro IPCA+ 2029,2029-05-15,7.41,3401.66
3,IPCA,Tesouro IPCA+ 2040,2040-08-15,7.09,1597.89
5,IPCA,Tesouro IPCA+ 2050,2050-08-15,6.99,828.62
15,IPCA,Tesouro IPCA+ com Juros Semestrais 2035,2035-05-15,7.31,4126.68
17,IPCA,Tesouro IPCA+ com Juros Semestrais 2045,2045-05-15,7.21,3973.75
20,IPCA,Tesouro IPCA+ com Juros Semestrais 2060,2060-08-15,7.14,3955.53
32,IPCA,Tesouro Renda+ Aposentadoria Extra 2030,2049-12-15,7.18,1781.75
33,IPCA,Tesouro Renda+ Aposentadoria Extra 2035,2054-12-15,7.10,1278.77
34,IPCA,Tesouro Renda+ Aposentadoria Extra 2040,2059-12-15,7.06,916.93
35,IPCA,Tesouro Renda+ Aposentadoria Extra 2045,2064-12-15,7.04,656.70


In [35]:
consultaTD('V','S')

,Tipo,Título,Vencimento,Rentabilidade (Venda),Preço R$ (Venda)
27,SELIC,Tesouro Selic 2026,2026-03-01,0.0141,16596.74
28,SELIC,Tesouro Selic 2027,2027-03-01,0.0475,16584.67
29,SELIC,Tesouro Selic 2028,2028-03-01,0.0702,16566.37
30,SELIC,Tesouro Selic 2029,2029-03-01,0.1088,16531.01
31,SELIC,Tesouro Selic 2031,2031-03-01,0.1216,16483.21


In [36]:
consultaTD('V','P')

,Tipo,Título,Vencimento,Rentabilidade (Venda),Preço R$ (Venda)
6,PREFIXADO,Tesouro Prefixado com Juros Semestrais 2027,2027-01-01,14.16,982.51
7,PREFIXADO,Tesouro Prefixado com Juros Semestrais 2029,2029-01-01,13.97,934.99
8,PREFIXADO,Tesouro Prefixado com Juros Semestrais 2031,2031-01-01,14.28,885.97
9,PREFIXADO,Tesouro Prefixado com Juros Semestrais 2033,2033-01-01,14.23,855.34
10,PREFIXADO,Tesouro Prefixado com Juros Semestrais 2035,2035-01-01,14.37,825.61
21,PREFIXADO,Tesouro Prefixado 2026,2026-01-01,14.81,917.55
22,PREFIXADO,Tesouro Prefixado 2027,2027-01-01,14.16,807.86
23,PREFIXADO,Tesouro Prefixado 2028,2028-01-01,13.78,714.21
24,PREFIXADO,Tesouro Prefixado 2029,2029-01-01,13.93,626.03
25,PREFIXADO,Tesouro Prefixado 2031,2031-01-01,14.16,477.64
